# Quantum KV-Cache Policy (Q-KV) - Interactive Exploration

This notebook explores Q-KV cache eviction policies vs classical baselines.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
from systems.q_kv_policy import QKVPolicy, LRUPolicy, LFUPolicy

## 1. Basic Cache Operations

In [ ]:
# Initialize policies
cache_size = 10
qkv = QKVPolicy(m=16, cache_size=cache_size)
lru = LRUPolicy(cache_size=cache_size)
lfu = LFUPolicy(cache_size=cache_size)

# Insert some keys
for i in range(5):
    key = f"key{i}".encode()
    value = f"value{i}"
    qkv.put(key, value)
    lru.put(key, value)
    lfu.put(key, value)

print(f"Q-KV cache size: {len(qkv.cache)}")
print(f"LRU cache size: {len(lru.cache)}")
print(f"LFU cache size: {len(lfu.cache)}")

## 2. Simulate Workload and Compare Hit Rates

In [ ]:
# Generate Zipf-distributed workload
def generate_zipf_workload(n_keys, n_accesses, alpha=1.5):
    keys = [f"key{i}".encode() for i in range(n_keys)]
    probs = 1 / np.arange(1, n_keys + 1) ** alpha
    probs /= probs.sum()
    return np.random.choice(keys, size=n_accesses, p=probs)

workload = generate_zipf_workload(50, 500)

# Reset policies
qkv.clear()
lru.clear()
lfu.clear()

# Process workload
for key in workload:
    for policy in [qkv, lru, lfu]:
        if policy.get(key) is None:
            policy.put(key, f"value_{key.decode()}")

print(f"Q-KV hit rate: {qkv.get_hit_rate():.3f}")
print(f"LRU hit rate: {lru.get_hit_rate():.3f}")
print(f"LFU hit rate: {lfu.get_hit_rate():.3f}")

## 3. TODO: Add hit rate plots and perplexity analysis